In [8]:
import pandas as pd
import numpy as np


In [11]:
df = pd.read_csv('spotify_millsongdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [12]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [13]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)


In [14]:
df['text'][0]

'She stood beside my narrow bed  \r\nTo check my E.K.G.  \r\nShe shook her pretty little head  \r\nAt what\'s become of me  \r\n  \r\nI thought I glimpsed a path that led  \r\nThrough rhododendron days  \r\nAnd fuchsia nights to the boat-shed  \r\nIn which we two once lay  \r\n  \r\nBut she gazed only at my chart  \r\nThe valleys and the peaks  \r\nBrought back the time she broke my heart  \r\nIn Macgillycuddy\'s Reeks  \r\n  \r\nBut she gazed only at my chart  \r\nThe valleys and the peaks  \r\nBrought back the time she broke my heart  \r\nIn Macgillycuddy\'s Reeks  \r\n  \r\nI saw her on Killarney\'s shore  \r\nOne morning in July  \r\nWhen I still thought I was a thorn  \r\nTrying to find a side  \r\n  \r\nI met her in the little launch  \r\nThat runs to Innisfallen  \r\nHunched together, haunch to haunch  \r\nTrying to keep my balance  \r\n  \r\nBut she upset my applecart  \r\nShe kissed me on the cheek  \r\nAnd I was struck by Cupid\'s dart  \r\nIn Macgillycuddy\'s Reeks  \r\n  \r

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\s+', ' ', regex=True).replace(r'\n', ' ', regex=True)


In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

# Create a stemmer
stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidVector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidVector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.01011854, 0.0022939 , ..., 0.00667406, 0.01087897,
       0.0345904 ])

In [23]:
def recommendation(song_df):
  if not df[df['song'] == song_df].empty:
        idx = df[df['song'] == song_df].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)

        return songs
  else:

        print(f"No data found for song: {song_df}")




In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))
